# Modelamiento no supervisado en base a tópicos

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

In [2]:
from utils.cargar import df_caso
from utils.preprocesamiento import process_df, procesar_adela

caso = 'adela'
df = df_caso(caso)

df = procesar_adela(df)
df = df[df['opt_left'] == 'Producir el alimento contra déficit vitamínico']

df = process_df(df,'comment','sel',verbose=True)

df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

26 rows found with non string elements for column comment (0.40%)
Deleting 685 columns for which max target value is over 7 (10.58%)
5761 available rows after processing


In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test, _, _ = train_test_split(df, df['sel'], test_size=.05, stratify=df['sel'], random_state=0)

In [4]:
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

In [5]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

Usamos el número de tópicos con mejor métrica de coherencia

In [6]:
%%time
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=4, id2word=dictionary, passes=10)

CPU times: user 25.2 s, sys: 5.63 ms, total: 25.2 s
Wall time: 25.2 s


In [7]:
# Print the generated topics
topics = lda_model.print_topics(num_topics=5)
for topic in topics:
    print(topic)

(0, '0.021*"tradiciones" + 0.017*"salud" + 0.013*"pueblos" + 0.013*"ser" + 0.011*"bien" + 0.011*"importante" + 0.010*"si" + 0.010*"originarios" + 0.007*"cultura" + 0.006*"personas"')
(1, '0.022*"tradiciones" + 0.014*"pueblo" + 0.012*"fruta" + 0.011*"puede" + 0.010*"importante" + 0.009*"si" + 0.009*"pueblos" + 0.008*"vitamina" + 0.008*"respetar" + 0.007*"llevar"')
(2, '0.022*"pueblo" + 0.017*"tradiciones" + 0.014*"si" + 0.013*"originario" + 0.011*"fruto" + 0.010*"importante" + 0.009*"acuerdo" + 0.009*"alimento" + 0.009*"producir" + 0.008*"llegar"')
(3, '0.030*"alimento" + 0.030*"tradiciones" + 0.023*"pueblo" + 0.015*"personas" + 0.014*"importante" + 0.014*"producir" + 0.012*"producción" + 0.011*"debería" + 0.011*"salud" + 0.011*"llegar"')


In [8]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
pyLDAvis.display(vis_data)

In [9]:
print(' '.join(tokenized_test[0]) + '\n')

# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

Se debe priorizar respetar gente pueblo insistir cantidad gente beneficiaria fruta llegar acuerdo beneficie bandos problema

Topic 0: 0.015024765394628048
Topic 1: 0.017109647393226624
Topic 2: 0.01526353694498539
Topic 3: 0.9526020288467407


## Correlaciones

In [10]:
from TM_utils import get_lda_embeddings
import pandas as pd

arr_test = get_lda_embeddings(lda_model,tokenizador,dictionary,df_test,'comment')
df_topics = pd.DataFrame(arr_test, columns=['Topic{}'.format(i+1) for i in range(len(topics))])

df_topics['sel'] = df_test['sel'].values

In [12]:
from scipy.stats import pearsonr

# Only compute pearson prod-moment correlations between feature
# columns and target column
target_col_name = 'sel'
feature_target_corr = {}
for col in df_topics:
    if target_col_name != col:
        feature_target_corr[col + '_' + target_col_name] = \
            pearsonr(df_topics[col], df_topics[target_col_name])[0]
print("Feature-Target Correlations")
print(feature_target_corr)

Feature-Target Correlations
{'Topic1_sel': -0.08857272474703744, 'Topic2_sel': 0.1729051360639037, 'Topic3_sel': 0.09872417184074367, 'Topic4_sel': -0.11057733770541987}
